- Update: 2023.02.25

# 0. 配置参数

- `dict_page`：品类链接的参数
- `out`：保存结果的路径

In [1]:
dict_page = {'url': 'https://www.cardone.com/search.php?page=WHAT_THE_FUCK&section=product&search_query=%22throttle+body%22&f=throttle+body&sort=relevance',
             'max_page': 36}

out = '../../file/cardone-throttle_body-menu.xlsx'

In [2]:
from selenium.webdriver import Chrome
import datetime

In [3]:
browser = Chrome()
browser.set_page_load_timeout(60)

print(f'开始爬取时间：{datetime.datetime.now().strftime("%Y:%m:%d %H:%M:%S")}')

开始爬取时间：2023:02:25 08:48:29


In [4]:
from selenium.webdriver.common.by import By
import time

In [5]:
col_url = []
col_title = []
col_part_number = []
col_vehicle = []
for i in range(1, dict_page['max_page']+1):
    print(i)
    browser.get(dict_page['url'].replace('WHAT_THE_FUCK', str(i)))
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(1.5)
    browser.execute_script('window.scrollTo(0, -document.body.scrollHeight)')
    time.sleep(0.5)
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(1.5)
    browser.execute_script('window.scrollTo(0, -document.body.scrollHeight)')
    time.sleep(0.5)
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(1)
    
    list_article = browser.find_elements(by=By.CLASS_NAME, value='card--cart-enabled')
    for article in list_article:
        content = article.find_element(by=By.CLASS_NAME, value='col1')
        title = content.find_element(by=By.TAG_NAME, value='h4').text
        col_title.append(title)
        #print(title)
        url = content.find_element(by=By.TAG_NAME, value='h4').find_element(by=By.TAG_NAME, value='a').get_attribute('href')
        col_url.append(url)
        # print(url)
        part_number = content.find_elements(by=By.TAG_NAME, value='p')[1].text.split('Part Number: ')[1]
        col_part_number.append(part_number)
        # print(part_number)
        list_li = article.find_element(by=By.CLASS_NAME, value='card-attributes').find_elements(by=By.TAG_NAME, value='li')
        list_vehicle = []
        for li in list_li:
            list_vehicle.append(li.text)
        vehicle = '\n'.join(list_vehicle)
        col_vehicle.append(vehicle)
        # print(vehicle)
        # print()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36


In [6]:
browser.quit()
print(f'爬取结束时间：{datetime.datetime.now().strftime("%Y:%m:%d %H:%M:%S")}')

爬取结束时间：2023:02:25 08:54:45


In [7]:
import pandas as pd

In [8]:
df = pd.DataFrame({'标题': col_title,
                   '品牌号': col_part_number,
                   '车型': col_vehicle,
                   '链接': col_url})
df.to_excel(out, index=False)